In [100]:
import sys 
sys.path.insert(1,"../src/")

# Create dummy inputs and store in the correct model directory 
For the moment want to create dummy input data, and keep this constant such that it does not change. Use this data to verify that the hw + cpu run has the same output as the expected output.

In [119]:
import numpy as np
import os
import shutil

clear_data = False

# Create a directory with relative info
model_dict = {}

# Get the onnx model directory
md = "../models"
op_type = "Conv"

# Get the model hw directory where the driver file is located.
build_dir = "../build-KV260/"

# Get split names for created hw files
split_dirs = [s for s in os.listdir(os.path.join(build_dir,op_type))]

# Fill the model dict
for split_name in split_dirs:
    hw_base_folder = os.path.join(build_dir,op_type,split_name,"deploy")
    model_base_folder = os.path.join(md,op_type,split_name)
    if clear_data:
        shutil.rmtree(f"{model_base_folder}/data")
    ! ls {model_base_folder}
    model_dict[split_name] = {"model_dir": model_base_folder, "hw_dir": hw_base_folder}


data  partition_0.onnx	partition_1.onnx
data  partition_0.onnx	partition_1.onnx
data  partition_0.onnx	partition_1.onnx
data  partition_0.onnx	partition_1.onnx
data  partition_0.onnx	partition_1.onnx
data  partition_0.onnx	partition_1.onnx


In [120]:
# Create dummy data for the moment
for k,v in model_dict.items():
    sd = v["model_dir"] + "/data"
    if not os.path.exists(sd):
        os.mkdir(sd)
        x = np.random.random([1,3,224,224])
        np.save(f"{sd}/{k}_input.npy",x)
        v["data_dir"] = sd
    else:
        v["data_dir"] = sd

# Get golden output 
For each of the inputs get the expected output from the baseline model

In [121]:
from qonnx.core.modelwrapper import ModelWrapper
from onnxruntime_extensions import get_library_path
import onnxruntime as ort
from custom_ort_functions import set_multithreshold_default
# TODO: ADD THIS STEP TO THE CUSTOM_ORT_FUNCTIONS
from qonnx.transformation.double_to_single_float import DoubleToSingleFloat

# Load input streamlined model
mf = "../models/mobilenet_streamline.onnx"
model = ModelWrapper(mf)
model = model.transform(DoubleToSingleFloat())
set_multithreshold_default(model,mf.replace('.onnx',"_ort.onnx"))

# Set up a session for 
so = ort.SessionOptions()
so.register_custom_ops_library(get_library_path())
sess = ort.InferenceSession(mf.replace('.onnx',"_ort.onnx"), so)

for k,v in model_dict.items():
    data_dir = v["data_dir"]
    x = np.load(f"{data_dir}/{k}_input.npy")
    inp_dict = {inp_name: x.astype(np.float32)}
    _gld_res = sess.run([],inp_dict)
    np.save(f"{data_dir}/{k}_gld_output.npy",_gld_res)